Copyright (c) 2008 Idiap Research Institute, http://www.idiap.ch/
    
Written by Suhan Shetty <suhan.shetty@idiap.ch>,


### Motion Planning without task-parameters

In [1]:
%load_ext autoreload
%autoreload 2

import torch
torch.set_default_dtype(torch.float64)

import numpy as np
np.set_printoptions(2, suppress=True)
torch.set_printoptions(2, sci_mode=False)

import sys
sys.path.append('../')
from panda_kinematics import PandaKinematics
from ttgo import TTGO
from manipulator_utils import exp_space, test_robotics_task
from utils import Point2PointMotion, test_ttgo
from panda_cost_utils import PandaCost,SDF_Cost 
device = "cpu" #torch.device("cuda" if torch.cuda.is_available() else "cpu")from motion_generation_utils import Point2PointMotion,Point2PointMotionCostFcn

In [2]:
# import the environment (SDF and for graphics visualization in pybullet)
import sys
DATA_PATH = './data'
body_sphere_path = './data/sphere_setting.npy'
sdf_path = './data/sdf.npy'
urdf_path = './data/urdf/frankaemika_new/panda_arm.urdf'

In [4]:
pick_from = "shelf"

Load the model and the parameters

In [5]:
# Load the parameters from the training
w_goal,w_obst,w_orient, w_ee, w_control = (0,1,1,0,1)
b_goal,b_obst,b_orient, b_ee, b_control = (1,0.05,0.05,1,1)

print("w: ",w_goal,w_obst,w_orient, w_ee, w_control)
print("b: ",b_goal,b_obst,b_orient, b_ee, b_control)

d0_w = 50
K = 2
margin= 0.1
dt = 0.05 #model['dt'] #
basis = 'rbf'
d_type = 'uniform'

Rd_0 = torch.tensor([[ 0.7071,0.7071,0.], [0.,0.,1],[0.7071, -0.7071, 0.]]).to(device) # desired orientation
v_d = torch.tensor([0.,0.,1.]).to(device)


w:  0 1 1 0 1
b:  1 0.05 0.05 1 1


In [6]:
data_sdf = np.load('./data/sdf.npy', allow_pickle=True)[()]
sdf_matr = data_sdf['sdf_matr']  # SDF tensor
bounds = torch.tensor(data_sdf['bounds']).to(device) # Bound of the environment
env_bound = data_sdf['env_bound']  
shelf_bound = data_sdf['shelf_bound'] 
box_bound = data_sdf['box_bound'] 

sdf_tensor = torch.from_numpy(sdf_matr).to(device)
sdf_cost = SDF_Cost(sdf_tensor=sdf_tensor, domain=bounds, device=device)

data_keys = np.load('./data/sphere_setting.npy', allow_pickle=True)[()]# key_points
status_array = data_keys['status_array']
body_radius = data_keys['body_radius']
relative_pos = data_keys['relative_pos']

key_points_weight = torch.from_numpy(status_array).to(device)>0 # 8xMx1
key_points_weight[-1] = True # activates thesphere on the gripper (inactie by default)
key_points_margin = torch.from_numpy(body_radius).to(device) #
key_points_pos = torch.from_numpy(relative_pos).to(device)

key_points_margin[-1] = margin  # end-effector-upper part
key_points_margin[-1][-1]= margin # gripper

key_points = [key_points_pos, key_points_weight, key_points_margin]
panda = PandaKinematics(device=device, key_points_data=key_points)

In [7]:

##############################################################################
# Define the domain  and cost function

# domain of joint angles
n_joints=7

# Discretize weights
max_w= 1*panda.theta_max
min_w = 1*panda.theta_min

d_w_all =  [d0_w]*n_joints# np.linspace(50,100,n_joints)
d_w = [int(d_w_all[joint]) for joint in range(n_joints)]
domain = [torch.linspace(min_w[i],max_w[i],d_w[i]) for i in range(n_joints)]*K


# Desired orientation of ee at the via-point
theta_limits = [panda.theta_min, panda.theta_max]
p2p = Point2PointMotion(dt=dt, K=K, n=n_joints, basis=basis,
                            bounds=theta_limits, device=device)
pandaCost = PandaCost(p2p_motion=p2p,robot=panda, sdf_cost=sdf_cost,Rd_0=Rd_0, v_d=v_d,
                    b_obst=b_obst,b_goal=b_goal, b_ee =b_ee,
                    b_control=b_control, b_orient=b_orient, 
                    device=device)  



In [8]:
# For the via-point problem only (intial and final configurations)
# theta_0 = torch.tensor([ -0.5,  -1.,  0.5, -2.,  0.4,  2.,  1.]).reshape(1,-1).to(device) # nominal pose for picking from shelf
# theta_0 = torch.tensor([ 0.17, -0.99,  0.99, -1.86,  0.83,  1.15, -1.49]).reshape(1,-1).to(device)


Cost functions

In [9]:
def cost_all(x):
    return pandaCost.cost_j2j(x, theta_0, theta_1)



def cost(x):
    return cost_all(x)[:,0]

def pdf(x):
    return torch.exp(-cost(x)**2)



TT-Model

## Visualization

In [10]:
import pybullet_data
from panda_visualization_utils import *
import pybullet as p
from functools import partial

p.connect(p.GUI)
data = np.load(sdf_path, allow_pickle=True)[()]
sdf_matr = data['sdf_matr']  #SDF tensor
obstacles = data['obstacles'] #obstacles parameters
colors = [[0.8, 0.5, 0.5, 1]]*len(obstacles)
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet (np.zeros(3), np.zeros(3), obstacles, colors=colors)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
robot_urdf ='./data/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])
set_q_std = partial(set_q,robot_id, pb_joint_indices)
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()
pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand_joint')
alpha = np.deg2rad(52)
quat = p.getQuaternionFromAxisAngle((0,0,1),alpha)
p.resetBasePositionAndOrientation(robot_id, (0,0,0.05), quat)

In [1]:
ik_data = torch.load('.data/panda_ik_solutions.pickle') # load init and final configurations
test_theta = ik_data['theta']
test_x = ik_data['task']

NameError: name 'torch' is not defined

In [14]:
# Fit the TT-model
# with torch.no_grad():
s1 = np.random.randint(0,test_theta.shape[0]-1)
s2 = np.random.randint(0,test_theta.shape[0]-1)


theta_0 = test_theta[s1].reshape(1,-1).to(device)
theta_1 = test_theta[s2].reshape(1,-1).to(device)

tt_model = tt_utils.cross_approximate(fcn=pdf,  domain=[x.to(device) for x in domain], 
                        rmax=200, nswp=20, eps=1e-3, verbose=True, 
# Refine the discretization and interpolate the model
scale_factor = 10
site_list = torch.arange(len(domain))#len(domain_task)+torch.arange(len(domain_decision))
domain_new = tt_utils.refine_domain(domain=domain, 
                                    site_list=site_list,
                                    scale_factor=scale_factor, device=device)
tt_model_new = tt_utils.refine_model(tt_model=tt_model.to(device), 
                                    site_list=site_list,
                                    scale_factor=scale_factor, device=device)                        kickrank=5, device=device)


ttgo = TTGO(tt_model=tt_model_new, domain=domain_new, cost=cost,device=device)

    

cross device is cpu
Cross-approximation over a 14D domain containing 6.10352e+23 grid points:
Note: The algorithm converges as the ratio tt-new-norm/tt-old-norm settles to 1. For TTGO, the convergence is not important, just keep iterating as long as the ratio > 1
iter: 0 | tt-new-norm/tt-old-norm: 5.359e-08 | time:   0.4530 | largest rank:   1
iter: 1 | tt-new-norm/tt-old-norm: 1.075e+05 | time:   3.2866 | largest rank:   3
iter: 2 | tt-new-norm/tt-old-norm: 1.787e+00 | time:   6.2392 | largest rank:   3
iter: 3 | tt-new-norm/tt-old-norm: 4.067e-01 | time:   9.0105 | largest rank:   3
iter: 4 | tt-new-norm/tt-old-norm: 1.201e+00 | time:  11.8648 | largest rank:   3 <- max_iter was reached: 5
Did 46350 function evaluations, which took 11.19s (4142 evals/s)



For a random task find the decision variables (multiple solutions)

In [16]:


n_solutions= 10
n_samples_tt = 100
n_samples_rand= 1000
alpha=0.75; norm=1 ;

t1 = time.time()
samples_tt, samples_idx = ttgo.sample_tt(n_samples=n_samples_tt, alpha=alpha)
state_k_tt = ttgo.choose_top_k_sample(samples_tt,n_solutions)[0]
t2=time.time()

#Optimize
state_k_tt_opt = 1*state_k_tt
for i, state in enumerate(state_k_tt):
    state_k_tt_opt[i,:],results= ttgo.optimize(state,bound=True)
t3 = time.time()
             
c_tt =  cost_all(state_k_tt_opt)

print("Cost-mean-tt:",torch.mean(c_tt,dim=0))


Cost-mean-tt: tensor([0.25, 0.02, 0.01, 0.27])


From the decision variables construct the joint angle trajectory

In [17]:
n_joints=7
theta_bounds = [panda.min_config, panda.max_config]
p2p_ =Point2PointMotion(dt=0.001, K=K, n=n_joints,bounds=theta_bounds, basis=basis, device=device)

x = 1*state_k_tt_opt
w = x[:] # basis weights
theta_t = p2p_.gen_traj_p2p(theta_0.repeat(n_solutions,1),theta_1.repeat(n_solutions,1),w) # batch x time x joint 


print("Costs: ", cost(x))


Costs:  tensor([0.21, 0.32, 0.21, 0.26, 0.26, 0.31, 0.28, 0.23, 0.23, 0.24])


In [18]:
# print("Joint Violate: ", torch.sum(theta_t < panda.theta_min_robot) + torch.sum(theta_t > panda.theta_max_robot) )

Visualize the joint angle trajectory

In [19]:
time.sleep(3)
dt = 0.01
for k_ in range(n_solutions):
    set_q_std(theta_t[0,0,:])
    T = theta_t.shape[1]
    time.sleep(30*dt)
    for t in range(T):
        set_q_std(theta_t[k_,t,:])
        time.sleep(1*dt)
    time.sleep(10*dt)


KeyboardInterrupt: 